In [ ]:
import tensorflow as tf
from tensorflow import keras


In [ ]:
imdb = keras.datasets.imdb
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)


17464789/17464789 [==============================] - 0s 0us/step


In [ ]:
word_index = imdb.get_word_index()

word_index = {k:(v+3) for k,v in word_index.items()}
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2
word_index["<UNUSED>"] = 3

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

train_data = keras.preprocessing.sequence.pad_sequences(train_data,
                                                        value=word_index["<PAD>"],
                                                        padding='post',
                                                        maxlen=256)

test_data = keras.preprocessing.sequence.pad_sequences(test_data,
                                                       value=word_index["<PAD>"],
                                                       padding='post',
                                                       maxlen=256)


1641221/1641221 [==============================] - 0s 0us/step


In [ ]:
model = keras.Sequential([
    keras.layers.Embedding(10000, 16),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 16)          160000    
                                                                 
 global_average_pooling1d (  (None, 16)                0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 160289 (626.13 KB)
Trainable params: 160289 (626.13 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [ ]:
history = model.fit(train_data,
                    train_labels,
                    epochs=10,
                    batch_size=512,
                    validation_data=(test_data, test_labels),
                    verbose=1)


Epoch 1/10
49/49 [==============================] - 3s 33ms/step - loss: 0.6907 - accuracy: 0.6424 - val_loss: 0.6859 - val_accuracy: 0.6967
Epoch 2/10
49/49 [==============================] - 1s 29ms/step - loss: 0.6732 - accuracy: 0.7411 - val_loss: 0.6572 - val_accuracy: 0.7418
Epoch 3/10
49/49 [==============================] - 1s 25ms/step - loss: 0.6270 - accuracy: 0.7789 - val_loss: 0.5994 - val_accuracy: 0.7868
Epoch 4/10
49/49 [==============================] - 1s 28ms/step - loss: 0.5546 - accuracy: 0.8130 - val_loss: 0.5268 - val_accuracy: 0.8152
Epoch 5/10
49/49 [==============================] - 2s 36ms/step - loss: 0.4762 - accuracy: 0.8423 - val_loss: 0.4602 - val_accuracy: 0.8369
Epoch 6/10
49/49 [==============================] - 2s 46ms/step - loss: 0.4094 - accuracy: 0.8634 - val_loss: 0.4082 - val_accuracy: 0.8497
Epoch 7/10
49/49 [==============================] - 1s 25ms/step - loss: 0.3590 - accuracy: 0.8773 - val_loss: 0.3725 - val_accuracy: 0.8582
Epoch 8/10
49

In [ ]:
test_loss, test_acc = model.evaluate(test_data, test_labels)

print('Test accuracy:', test_acc)


782/782 [==============================] - 2s 2ms/step - loss: 0.3208 - accuracy: 0.8689
Test accuracy: 0.8689200282096863


In [ ]:
# Function to preprocess the user-entered review
def preprocess_review(review):
    review = review.lower()
    review = review.split()
    review = [word_index[word] if word in word_index and word_index[word] < 10000 else 2 for word in review]
    review = keras.preprocessing.sequence.pad_sequences([review],
                                                         value=word_index["<PAD>"],
                                                         padding='post',
                                                         maxlen=256)
    return review

# User enters a review
user_review = input("Enter a movie review: ")

# Preprocess the user-entered review
preprocessed_review = preprocess_review(user_review)

# Predict the class label for the review
prediction = model.predict(preprocessed_review)

# Print the predicted class label
if prediction[0][0] >= 0.5:
    print("Predicted class: Positive")
else:
    print("Predicted class: Negative")


Enter a movie review: PATHAAN IS REALLY NICE MOVIE. ITS WONDERFUL
1/1 [==============================] - 0s 22ms/step
Predicted class: Positive
